In [26]:
#!pip install opencv-python

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow import keras
from tensorflow.keras import layers
import os
import glob
import cv2

In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Splitting dataset (splitting paths leading to images)

## getting dataset from kaggle

In [3]:
! mkdir -p ~/.kaggle;
! cp kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [30]:
! kaggle datasets download -d andradaolteanu/gtzan-dataset-music-genre-classification

gtzan-dataset-music-genre-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [31]:
! unzip gtzan-dataset-music-genre-classification.zip

Archive:  gtzan-dataset-music-genre-classification.zip
replace Data/features_30_sec.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Data/features_3_sec.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Data/genres_original/blues/blues.00000.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Data/genres_original/blues/blues.00001.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## creating dataset with paths to spectograms

In [3]:
# specify path to spectogram images
PATH = r"/content/Data/images_original/"

In [4]:
genres = os.listdir(PATH)

In [5]:
# getting path of all file names in directories
files = {}
for genre in genres:
    files[genre] = glob.glob(f"{PATH}/{genre}/*png")

In [6]:
# creating dataframe wit paths
df = pd.DataFrame(columns = ['path', 'rgb_data', 'label'])
for genre in genres:
    df_aux = pd.DataFrame(files[genre], columns = ['path'])
    df_aux['label'] = genre
    df = df.append(df_aux, ignore_index=True, sort=False)

# verifying number of rows
df['label'].value_counts()

metal        100
reggae       100
country      100
rock         100
blues        100
disco        100
pop          100
classical    100
hiphop       100
jazz          99
Name: label, dtype: int64

In [7]:
#getting rgb data for all images
for index in df.index:
    img = cv2.imread(df['path'].loc[index])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #df['rgb_data'].loc[index] = np.array(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), dtype=np.uint8)
    df['rgb_data'].loc[index] = np.asarray(img).astype('float32')

In [8]:
df.head()

,path,rgb_data,label
0,/content/Data/images_original//classical/class...,"[[[255.0, 255.0, 255.0], [255.0, 255.0, 255.0]...",classical
1,/content/Data/images_original//classical/class...,"[[[255.0, 255.0, 255.0], [255.0, 255.0, 255.0]...",classical
2,/content/Data/images_original//classical/class...,"[[[255.0, 255.0, 255.0], [255.0, 255.0, 255.0]...",classical
3,/content/Data/images_original//classical/class...,"[[[255.0, 255.0, 255.0], [255.0, 255.0, 255.0]...",classical
4,/content/Data/images_original//classical/class...,"[[[255.0, 255.0, 255.0], [255.0, 255.0, 255.0]...",classical


In [9]:
from sklearn.preprocessing import LabelEncoder

In [10]:
# encoding label
le = LabelEncoder()
le.fit(genres)
df['label_encoded'] = le.transform(df['label'])

In [11]:
df['label_encoded']

0      1
1      1
2      1
3      1
4      1
      ..
994    4
995    4
996    4
997    4
998    4
Name: label_encoded, Length: 999, dtype: int64

## splitting dataset

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
#defining X and y
X = [data for data in df['rgb_data']]
X = np.array(X)
y = np.array(df['label_encoded'])

#splitting data and stratifying by y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Modelling

In [14]:
shape_of_input = df['rgb_data'][0].shape
shape_of_input

(288, 432, 3)

In [15]:
model = keras.Sequential([
    layers.Rescaling(1 / 255, input_shape=shape_of_input),
    
    layers.Conv2D(8, kernel_size=(3, 3), strides=(1, 1), activation='relu'),
    layers.BatchNormalization(axis=3),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(16, kernel_size=(3, 3), strides=(1, 1), activation='relu'),
    layers.BatchNormalization(axis=3),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'),
    layers.BatchNormalization(axis=3),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, kernel_size=(3, 3), strides=(1, 1), activation='relu'),
    layers.BatchNormalization(axis=-1),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(128, kernel_size=(3, 3), strides=(1, 1), activation='relu'),
    layers.BatchNormalization(axis=-1),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    
    layers.Dropout(rate=0.3),
    
    layers.Dense(len(genres), activation='softmax')
])

In [16]:
print(f'Model input shape: {model.input_shape}, Model output shape: {model.output_shape}')

Model input shape: (None, 288, 432, 3), Model output shape: (None, 10)


In [17]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 288, 432, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 286, 430, 8)       224       
_________________________________________________________________
batch_normalization (BatchNo (None, 286, 430, 8)       32        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 143, 215, 8)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 141, 213, 16)      1168      
_________________________________________________________________
batch_normalization_1 (Batch (None, 141, 213, 16)      64        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 70, 106, 16)       0

In [18]:
from keras.callbacks import EarlyStopping
# defining call back
callback = EarlyStopping(monitor="val_loss", patience=150, verbose=1, restore_best_weights=True)

In [19]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = 'accuracy')
model.fit(X_train, y_train, validation_split=0.1, batch_size=16, epochs=1000, callbacks=[callback])

Epoch 1/1000
45/45 [==============================] - 6s 80ms/step - loss: 4.2698 - accuracy: 0.2615 - val_loss: 2.7561 - val_accuracy: 0.0875
Epoch 2/1000
45/45 [==============================] - 3s 64ms/step - loss: 2.0151 - accuracy: 0.4979 - val_loss: 4.3841 - val_accuracy: 0.0625
Epoch 3/1000
45/45 [==============================] - 3s 64ms/step - loss: 2.2505 - accuracy: 0.5174 - val_loss: 4.7611 - val_accuracy: 0.0750
Epoch 4/1000
45/45 [==============================] - 3s 62ms/step - loss: 1.1149 - accuracy: 0.6912 - val_loss: 5.6354 - val_accuracy: 0.1875
Epoch 5/1000
45/45 [==============================] - 3s 63ms/step - loss: 0.6508 - accuracy: 0.7858 - val_loss: 6.3686 - val_accuracy: 0.0750
Epoch 6/1000
45/45 [==============================] - 3s 62ms/step - loss: 0.4841 - accuracy: 0.8456 - val_loss: 4.6103 - val_accuracy: 0.1625
Epoch 7/1000
45/45 [==============================] - 3s 71ms/step - loss: 0.2915 - accuracy: 0.9054 - val_loss: 4.2512 - val_accuracy: 0.2625

In [22]:
evaluation = model.evaluate(X_test, y_test)
print(f'Accuracy achieved on test set: {evaluation[1]:.4f}')

7/7 [==============================] - 2s 108ms/step - loss: 3.4808 - accuracy: 0.5650
Accuracy achieved on test set: 0.5650
